In [2]:
from tensorflow.keras.models import model_from_json
from convert_data import Idx

import cv2
import numpy as np
import matplotlib.pyplot as plt
import scipy.misc
import imageio

from sklearn.preprocessing import StandardScaler

In [4]:
model_path = "./model/model.json"
weights_path = "./model/model.h5"

# Load the model from file
model_file = open(model_path, 'r')
model = model_file.read()
model = model_from_json(model)
model_file.close()

# Load the weights from file and add them to the model
model.load_weights(weights_path)
print("Model and weights loaded")

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model and weights loaded


In [5]:
def convert_image(img, blur=3):
    # Convert to grayscale
    conv_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Adaptive thresholding to binarize the image
    conv_img = cv2.adaptiveThreshold(conv_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 4)

    # Blur the image to reduce noise
    conv_img = cv2.medianBlur(conv_img, blur)
    
    return conv_img

In [6]:
def extract_char(conv_img):
    # Find contours
    _, ctrs, _ = cv2.findContours(conv_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours
    return ctrs

In [9]:
# Turn camera on
cap = cv2.VideoCapture(0)

In [8]:
conv_size = [80, 80]
class_names = ['2', '5', '6', '8', 'a', 'A', 'b', 'B', 'c', 'ch', 'ck', 'd', 'D', 
               'e', 'E', 'f', 'F', 'g', 'G', 'h', 'i', 'k', 'K', 'l', 'm', 'M', 'n', 'N', 'o', 
               'p', 'P', 'r', 's', 'S', 't', 'T', 'tz', 'u', 'v', 'w', 'W', 'z', 'Z', 'ß']
while(1):
    # Take each frame
    _, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    conv_img = convert_image(frame, blur=3)
    
    sorted_ctrs =extract_char(conv_img)
        
    # Add bounding boxes
    n = 0
    nChar = np.shape(sorted_ctrs)[0]
    for i in range(nChar):
        # Draw the bounding box
        x, y, w, h = cv2.boundingRect(sorted_ctrs[i])
        if (w * h) > 600 and (w * h) < 20000:
            
            roi = gray[y:y+h, x:x+w]
            roi = cv2.resize(roi, (conv_size[0], conv_size[1]))
            roi = roi.reshape(1, conv_size[0], conv_size[1])
            roi = 255 - roi 
            
            prediction = model.predict(roi)
        
            if prediction.max() > 0.9:
                char = class_names[prediction.argmax()]

                im = cv2.putText(
                    frame, #numpy array on which text is written
                    char, #text
                    (x, y), #position at which writing has to start
                    cv2.FONT_HERSHEY_SIMPLEX, #font family
                    0.5, #font size
                    (209, 80, 0, 255), #font color
                    2) #font stroke

            n += 1
    
    cv2.imshow('frame', frame)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break

cv2.destroyAllWindows()